In [35]:
from openai import AzureOpenAI
import os
import json
import multiprocessing
import sys
sys.path.append('../src/')
from utils import generate_response, get_summary

In [36]:
# load demonstration data
demo_file = '/Users/Pengyun_Wang/Projects/HSBC/data/demonstration.json'
with open(demo_file, 'r') as f:
    demo_data = json.load(f)

# load web data
web_file = '/Users/Pengyun_Wang/Projects/HSBC/data/popular_web_data.json'
with open(web_file, 'r') as f:
    web_data = json.load(f)


In [37]:

def construct_prompt(demo_data, query):
    """
    Construct a prompt for the model to generate a human-readable summary of the website structure.
    Few shot learning is used to provide the model with context about the task.
    
    """
    start_phrase = f'''
    Task: Use the following examples to generate a human-readable summary of the website structure. Make sure to follow the examples and provide a step by step descritption. Most importantly, make sure to provide a stand-alone overall summary as the last paragraph.
    Prompt:
    Summarize the DOM data in human-readable way:
    '''
    # add demo
    for _, val in demo_data.items():
        start_phrase +=f"DOM: {val['web_data']}\n"
        start_phrase +=f"Summary: {val['summary']}\n"
    # add query
    start_phrase += f"DOM: {query}\n"
    start_phrase += f"Summary:"

    return start_phrase



In [44]:
# generate openai response with multiprocesses
prompts = [construct_prompt(demo_data, web_data[key]) for key in web_data.keys()]

with multiprocessing.Pool() as pool:
    responses = pool.map(generate_response, prompts)

Error code: 400 - {'error': {'message': "This model's maximum context length is 8193 tokens, however you requested 10191 tokens (9191 in your prompt; 1000 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error code: 400 - {'error': {'message': "This model's maximum context length is 8193 tokens, however you requested 10074 tokens (9074 in your prompt; 1000 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error code: 400 - {'error': {'message': "This model's maximum context length is 8193 tokens, however you requested 9206 tokens (8206 in your prompt; 1000 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error code: 400 - {'error': {'message': "This model's maximum context length is 8193 tokens, however you requested 10997 tokens (99

In [45]:
# save responses
output = {}
for i, key in enumerate(web_data.keys()):
    if responses[i] is not None:
        output[key] = responses[i]

with open('/Users/Pengyun_Wang/Projects/HSBC/data/openai_responses.json', 'w') as f:
    json.dump(output, f)
    
print(f"Number of successful responses: {len(output)}, Number of failed responses: {len(prompts) - len(output)}")

Responses saved to /Users/Pengyun_Wang/Projects/HSBC/data/openai_responses.json
Number of successful responses: 850, Number of failed responses: 34


In [46]:
# save the overall summary part in the response

    
overall_summary = {}
for key, val in output.items():
    tmp = get_summary(val)
    if tmp:
        overall_summary[key] = tmp
    
with open('/Users/Pengyun_Wang/Projects/HSBC/data/overall_summary.json', 'w') as f:
    json.dump(overall_summary, f)

print(f"Number of successful overall summaries: {len(overall_summary)}, Number of failed overall summaries: {len(output) - len(overall_summary)}")

Number of successful overall summaries: 843, Number of failed overall summaries: 7
